In [2]:
# imports
import requests
import pandas as pd
import numpy as np
import datetime

This notebook was used to research the code need to get the weather forecast data to be able to give future predictions on bike availability

Two foresecasts are available from the open weather maps API:

- 2 day forecast, with 1 hour intervals
- 5 day forecast, with 3 hour intervals

In [4]:
# get 2 day forecast, data is at 1 hour intervals
weatherTwoDay = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat=53.346&lon=6.26986&exclude=current,minutely&appid=98310ef86bbb250277915291623ed079")
twoDayJSON = weatherTwoDay.json()
twoDayForecast = pd.json_normalize(twoDayJSON, record_path =['hourly'])
twoDayForecast = twoDayForecast[['temp', 'wind_speed', 'humidity', 'dt']]
twoDayForecast['dt'] = pd.to_datetime(twoDayForecast['dt'], unit='s')

# get 5 day forecast, data is at 3 hour intervals
weatherFiveDay = requests.get("http://api.openweathermap.org/data/2.5/forecast?lat=53.346&lon=-6.26986&appid=98310ef86bbb250277915291623ed079")
fiveDayJSON = weatherFiveDay.json()
fiveDayForecast = pd.json_normalize(fiveDayJSON, record_path =['list'])
fiveDayForecast = fiveDayForecast[['main.temp','wind.speed', 'main.humidity', 'dt']]
forcast_fiveday = pd.json_normalize(fiveDayJSON, record_path = ['list', 'weather'])['main']
fiveDayForecast = pd.concat([forcast_fiveday, fiveDayForecast], axis=1)
fiveDayForecast = fiveDayForecast.rename(columns={"main.temp": "temp", "wind.speed": "wind_speed", "main.humidity": "humidity"})
fiveDayForecast['dt'] = pd.to_datetime(fiveDayForecast['dt'], unit='s')

In [5]:
twoDayForecast.head()

,temp,wind_speed,humidity,dt
0,281.18,5.24,70,2021-04-14 11:00:00
1,281.51,5.52,72,2021-04-14 12:00:00
2,281.23,5.89,70,2021-04-14 13:00:00
3,280.92,6.03,68,2021-04-14 14:00:00
4,280.47,6.11,67,2021-04-14 15:00:00


In [6]:
fiveDayForecast.head()

,main,temp,wind_speed,humidity,dt
0,Clouds,283.75,3.17,62,2021-04-14 12:00:00
1,Clouds,283.66,2.66,63,2021-04-14 15:00:00
2,Clouds,282.40,2.36,71,2021-04-14 18:00:00
3,Clear,277.99,1.58,90,2021-04-14 21:00:00
4,Clear,277.46,1.70,90,2021-04-15 00:00:00


## interpolating data

Our model is trained at 30 minute intervals, so we need to interpolate the forescast data to have it too at 30 min intervals

The original plan had been to use the 2 day forecast for the next 2 days, and the 5 day forecast for the remaining 3. However, it turns out the 2 day and 5 day forecasts are very different for the same timestamps, so rather than collating the data as I had planned I will intead just interpolate rows on the 5 day forecast to get a forecast at 30 min intervals

In [7]:
thirtyMinuteDF = pd.DataFrame(columns=['main', 'temp', 'wind_speed', 'humidity', 'dt'])

for index, row in fiveDayForecast.iterrows():
    df = pd.DataFrame([[row['main'], row['temp'], row['wind_speed'], row['humidity'], row['dt']]], columns=['main', 'temp', 'wind_speed', 'humidity', 'dt'])
    thirtyMinuteDF = thirtyMinuteDF.append(df, ignore_index=True)
    for i in range(30, 180, 30):
        new_time = row['dt'] + datetime.timedelta(minutes=i)
        df = pd.DataFrame([[row['main'], np.nan, np.nan, np.nan, new_time]], columns=['main', 'temp', 'wind_speed', 'humidity', 'dt'])
        thirtyMinuteDF = thirtyMinuteDF.append(df, ignore_index=True)
        

There was an issue with type of 'humidity' column here, preventing interpolating. so it has to be set to numeric

In [8]:
thirtyMinuteDF.dtypes

main                  object
temp                 float64
wind_speed           float64
humidity              object
dt            datetime64[ns]
dtype: object

In [9]:
thirtyMinuteDF['humidity'] = pd.to_numeric(thirtyMinuteDF['humidity'])

In [10]:
thirtyMinuteDF['temp'] = thirtyMinuteDF['temp'].interpolate()
thirtyMinuteDF['wind_speed'] = thirtyMinuteDF['wind_speed'].interpolate()
thirtyMinuteDF['humidity'] = thirtyMinuteDF['humidity'].interpolate()

In [11]:
thirtyMinuteDF = thirtyMinuteDF.round({'temp': 2, 'wind_speed': 2, 'humidity': 1})

In [12]:
thirtyMinuteDF

,main,temp,wind_speed,humidity,dt
0,Clouds,283.75,3.17,62.0,2021-04-14 12:00:00
1,Clouds,283.74,3.08,62.2,2021-04-14 12:30:00
2,Clouds,283.72,3.00,62.3,2021-04-14 13:00:00
3,Clouds,283.71,2.92,62.5,2021-04-14 13:30:00
4,Clouds,283.69,2.83,62.7,2021-04-14 14:00:00
...,...,...,...,...,...
235,Rain,282.93,1.59,96.0,2021-04-19 09:30:00
236,Rain,282.93,1.59,96.0,2021-04-19 10:00:00
237,Rain,282.93,1.59,96.0,2021-04-19 10:30:00
238,Rain,282.93,1.59,96.0,2021-04-19 11:00:00


### encoding data

Next we need to encode the data so that it matches the training data used

In [13]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
categories = np.array(['Clear', 'Clouds', 'Drizzle', 'Mist', 'Rain', 'Snow']).reshape(-1,1)
type_encoder = OneHotEncoder().fit(categories)
type_encoded = type_encoder.transform(np.array(thirtyMinuteDF["main"]).reshape(-1,1))
type_encoded = pd.DataFrame(type_encoded.toarray(), columns = [category for category in type_encoder.categories_[0]])
temp = thirtyMinuteDF.reset_index(drop=True)
encodedThirtyMinuteDF = pd.concat([type_encoded, temp[["temp", "wind_speed", "humidity", "dt"]]], axis = 1)

In [15]:
encodedThirtyMinuteDF

,Clear,Clouds,Drizzle,Mist,Rain,Snow,temp,wind_speed,humidity,dt
0,0.0,1.0,0.0,0.0,0.0,0.0,283.75,3.17,62.0,2021-04-14 12:00:00
1,0.0,1.0,0.0,0.0,0.0,0.0,283.74,3.08,62.2,2021-04-14 12:30:00
2,0.0,1.0,0.0,0.0,0.0,0.0,283.72,3.00,62.3,2021-04-14 13:00:00
3,0.0,1.0,0.0,0.0,0.0,0.0,283.71,2.92,62.5,2021-04-14 13:30:00
4,0.0,1.0,0.0,0.0,0.0,0.0,283.69,2.83,62.7,2021-04-14 14:00:00
...,...,...,...,...,...,...,...,...,...,...
235,0.0,0.0,0.0,0.0,1.0,0.0,282.93,1.59,96.0,2021-04-19 09:30:00
236,0.0,0.0,0.0,0.0,1.0,0.0,282.93,1.59,96.0,2021-04-19 10:00:00
237,0.0,0.0,0.0,0.0,1.0,0.0,282.93,1.59,96.0,2021-04-19 10:30:00
238,0.0,0.0,0.0,0.0,1.0,0.0,282.93,1.59,96.0,2021-04-19 11:00:00


In [16]:
encodedThirtyMinuteDF["dayOfWeek"] = encodedThirtyMinuteDF["dt"].dt.weekday
encodedThirtyMinuteDF["hour"] = encodedThirtyMinuteDF["dt"].dt.hour
encodedThirtyMinuteDF["minute"] = encodedThirtyMinuteDF["dt"].dt.minute


In [17]:
encodedThirtyMinuteDF.head()

,Clear,Clouds,Drizzle,Mist,Rain,Snow,temp,wind_speed,humidity,dt,dayOfWeek,hour,minute
0,0.0,1.0,0.0,0.0,0.0,0.0,283.75,3.17,62.0,2021-04-14 12:00:00,2,12,0
1,0.0,1.0,0.0,0.0,0.0,0.0,283.74,3.08,62.2,2021-04-14 12:30:00,2,12,30
2,0.0,1.0,0.0,0.0,0.0,0.0,283.72,3.00,62.3,2021-04-14 13:00:00,2,13,0
3,0.0,1.0,0.0,0.0,0.0,0.0,283.71,2.92,62.5,2021-04-14 13:30:00,2,13,30
4,0.0,1.0,0.0,0.0,0.0,0.0,283.69,2.83,62.7,2021-04-14 14:00:00,2,14,0


In [18]:
encodedThirtyMinuteDF = encodedThirtyMinuteDF[["Clear","Clouds", "Drizzle","Mist","Rain","Snow","dayOfWeek","hour","minute","temp","humidity","wind_speed"]]

In [19]:
encodedThirtyMinuteDF.head()

,Clear,Clouds,Drizzle,Mist,Rain,Snow,dayOfWeek,hour,minute,temp,humidity,wind_speed
0,0.0,1.0,0.0,0.0,0.0,0.0,2,12,0,283.75,62.0,3.17
1,0.0,1.0,0.0,0.0,0.0,0.0,2,12,30,283.74,62.2,3.08
2,0.0,1.0,0.0,0.0,0.0,0.0,2,13,0,283.72,62.3,3.00
3,0.0,1.0,0.0,0.0,0.0,0.0,2,13,30,283.71,62.5,2.92
4,0.0,1.0,0.0,0.0,0.0,0.0,2,14,0,283.69,62.7,2.83
